# **분류 모델과 트리 모델**

- <a href="#분류">분류(Classification)</a>
- <a href="#로지스틱 회귀">로지스틱 회귀(Logistic Regression)</a>
- <a href="#의사결정나무">의사결정나무(Decision tree)</a>
- <a href="#랜덤포레스트">랜덤포레스트(Random Forest)</a>
- <a href="#그라디언트부스팅">그라디언트 부스팅(gradient boosting)</a>

--------------

## <a name="분류">분류(Classification)</a>

### **예제 1: 타이타닉 생존 예측 (이진 분류)**

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 간단한 타이타닉 데이터 생성 (실제로는 kaggle에서 다운로드)
data = {
    'Pclass': [1, 2, 3, 1, 2, 3, 1, 2, 3, 1],
    'Sex': ['male', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'male', 'female'],
    'Age': [25, 35, 20, 28, 40, 22, 35, 27, 45, 30],
    'Survived': [1, 1, 0, 1, 0, 1, 1, 1, 0, 1]
}

df = pd.DataFrame(data)

# 성별을 숫자로 변환
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex']) # female: 0, male: 1

# 특성과 타겟 분리
X = df[['Pclass', 'Sex', 'Age']]
y = df['Survived']

# 로지스틱 회귀 모델
log_model = LogisticRegression()
log_model.fit(X, y)


# 예측 확률과 최종 예측 결과를 미리 계산
probabilities = log_model.predict_proba(X)
binary_predictions = log_model.predict(X)

print("--- 승객별 생존 확률 및 최종 예측 결과 ---")
# 하나의 반복문으로 각 승객의 확률과 예측 결과를 함께 출력
for i in range(len(X)):
    # 생존 확률 (1일 확률)
    survival_prob = probabilities[i][1]
    # 최종 예측 (0 또는 1)
    prediction = binary_predictions[i]
    # 실제 값
    actual = y[i]
    # 예측과 실제 값 비교
    result_icon = "✅" if prediction == actual else "❌"

    print(f"승객 {i+1}: 생존 확률 {survival_prob:.2f}  ->  예측: {prediction} (실제: {actual}) {result_icon}")

print("-" * 40)


# 성능 평가를 위한 혼동 행렬(Confusion Matrix) 시각화
print("\n--- 모델 예측 성능 시각화 ---")
cm = confusion_matrix(y, binary_predictions)

plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['death (0)', 'survival (1)'],
            yticklabels=['death (0)', 'survival (1)'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

### **예제 2: 텍스트 감정 분석 (Multi-class)**
- 참고 : https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import numpy as np

# 1. 데이터 로딩 및 준비
# Kaggle에서 다운로드한 데이터를 불러옵니다.
# 데이터는 세미콜론(;)으로 구분되어 있고, 헤더가 없습니다.
try:
    df_train = pd.read_csv('multi-class-train.txt', sep=';', header=None, names=['text', 'emotion'])
    df_val = pd.read_csv('multi-class-val.txt', sep=';', header=None, names=['text', 'emotion'])

    # 훈련 데이터와 검증 데이터를 합쳐서 전체 학습 데이터로 사용합니다.
    df_full = pd.concat([df_train, df_val], ignore_index=True)

    print("--- 데이터 로딩 완료 ---")
    print(f"전체 데이터 샘플 수: {len(df_full)}")
    print("감정 클래스 분포:")
    print(df_full['emotion'].value_counts())
    print("-" * 30)

except FileNotFoundError:
    print("오류: 'train.txt' 또는 'val.txt' 파일이 없습니다.")
    print("Kaggle에서 데이터셋을 다운로드하여 스크립트와 같은 폴더에 저장해주세요.")
    exit()

# 2. 머신러닝 파이프라인(Pipeline) 구축
# Pipeline은 데이터 전처리 단계와 모델 학습 단계를 하나로 묶어주는 편리한 기능입니다.
# 1단계: TfidfVectorizer - 텍스트 데이터를 TF-IDF 벡터로 변환합니다.
# 2단계: LogisticRegression - 다중 클래스 분류를 위한 로지스틱 회귀 모델을 사용합니다.
text_clf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, stop_words='english')),
    ('clf', LogisticRegression(solver='liblinear', multi_class='auto', random_state=42))
])

# 3. 모델 학습
# Pipeline을 사용하여 텍스트 데이터(X)와 감정 라벨(y)을 학습시킵니다.
X_train = df_full['text']
y_train = df_full['emotion']

print("\n--- 모델 학습 시작 ---")
text_clf_pipeline.fit(X_train, y_train)
print("모델 학습 완료!")
print("-" * 30)


# 4. 다중 분류 테스트를 위한 함수 및 실행
def predict_emotion(text):
    """
    입력된 텍스트의 감정을 예측하고, 각 감정에 대한 확률을 반환하는 함수
    """
    # 예측 (가장 확률이 높은 감정)
    predicted_emotion = text_clf_pipeline.predict([text])[0]

    # 각 클래스(감정)에 대한 예측 확률
    probabilities = text_clf_pipeline.predict_proba([text])[0]

    # 클래스(감정) 이름 가져오기
    emotion_classes = text_clf_pipeline.classes_

    # 각 감정과 확률을 딕셔너리로 묶어서 반환
    emotion_probabilities = {emotion: prob for emotion, prob in zip(emotion_classes, probabilities)}

    return predicted_emotion, emotion_probabilities

# 사용자가 직접 문장을 입력하여 테스트하는 무한 루프
print("\n--- 📝 텍스트 감정 분석기 (종료하려면 'exit' 입력) ---")
while True:
    try:
        user_input = input("분석할 문장을 입력하세요: ")
        if user_input.lower() == 'q':
            print("프로그램을 종료합니다.")
            break

        predicted, probs = predict_emotion(user_input)

        print(f"\n▶️ 예측된 감정: {predicted.upper()}")
        print("   [각 감정별 확률]")
        for emotion, prob in sorted(probs.items(), key=lambda item: item[1], reverse=True):
            print(f"     - {emotion:10s}: {prob:.4f} ({prob*100:6.2f}%)")
        print("-" * 30)

    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

### 예제 3: 텍스트 분류 (Multi-label)
- 참고 : https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/data

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

# 1. 데이터 로드
# train.csv 파일에는 'comment_text'와 여러 악성 유형 라벨 컬럼들이 있습니다.
df = pd.read_csv('multi-label-train.csv')

# 라벨 컬럼들만 따로 지정
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df['none'] = 1 - df[label_cols].max(axis=1) # 아무 유형에도 속하지 않는 'clean' 댓글 라벨 추가

# 2. 텍스트 데이터 전처리 및 벡터화 (TF-IDF)
# 댓글(text)을 컴퓨터가 이해할 수 있는 숫자 벡터로 변환합니다.
# TfidfVectorizer는 단어의 중요도를 계산하여 벡터를 만듭니다.
vectorizer = TfidfVectorizer(max_features=20000, stop_words='english')
X = vectorizer.fit_transform(df['comment_text']) # 훈련 데이터
y = df[label_cols] # 훈련 데이터의 라벨

# 3. 모델 학습
# OneVsRestClassifier를 사용해 각 라벨별로 Logistic Regression 모델을 학습시킵니다.
# 즉, 'toxic'인지 아닌지 판단하는 모델, 'obscene'인지 아닌지 판단하는 모델 등을 각각 만듭니다.
model = OneVsRestClassifier(LogisticRegression(solver='liblinear'), n_jobs=-1)
model.fit(X, y)

# 4. 예측
# 새로운 댓글 데이터가 있다면, 동일한 vectorizer로 변환 후 예측합니다.
new_comment = ["This is a friendly and helpful comment."]
new_comment_vec = vectorizer.transform(new_comment)

prediction = model.predict_proba(new_comment_vec) # 각 라벨에 대한 확률 예측

# 결과 출력 (예시)
# [[0.08, 0.01, 0.03, 0.00, 0.04, 0.01]] -> 각 라벨에 속할 확률이 낮음
print(pd.DataFrame(prediction, columns=label_cols))



---



## <a name="로지스틱 회귀">로지스틱 회귀(Logistic Regression)</a>

-  API : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html  

### 1) 로지스틱 회귀 이해하기

#### (1) 시그모이드 함수 만들기

In [ ]:
# numpy 라이브러리 및 그래프 라이브러리 불러오기
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Sigmoid 함수 작성하기
def sigmoid(x):
    # nnumpy.exp() 함수는 밑이 자연상수 e인 지수함수(e^x)로 변환
    return 1 / (1 + np.exp(-x))

In [ ]:
# 함수 테스트용 데이터 생성하기
test = np.array([-1, 0, 1])

In [ ]:
# 작성된 함수 확인하기
print(sigmoid(test))

#### (2) 시그모이드 함수 그래프 그리기

In [ ]:
# 그래프 적용을 위한 데이터 만들기
sigmoid_x = range(-6, 7)
sigmoid_y = sigmoid(np.array(sigmoid_x))

In [ ]:
# 선 그래프 그리기
plt.plot(sigmoid_x , sigmoid_y, color = 'blue',linewidth = 0.5)
# 백 그라운드 모눈 종이 설정하기
plt.rcParams['axes.grid'] = True
# 라인 굵기 설정하기
plt.axvline(x=0, color='black', linewidth=3)
# y 축 범위 설정하기
plt.yticks([0,0.5,1])
plt.show()

### 2) 로지스틱 회귀 실습하기

#### (1) 데이터 구성하기

In [ ]:
# 학습 데이터 생성하기
x_train = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
y_train = [0,0,0,0,0,0,0,1,1,1,1,1,1,1,1]
# 추론을 위한 데이터 생성하기
x_test = [0,1,2,18,19]
y_test =  [0,0,0,1,1]

# 학습 데이터에 대해 numpy로 변경 및 행을 열로 변경하기
x_train = np.array(x_train).reshape([-1,1])
y_train = np.array(y_train)
# 추론 데이터에 대해 numpy로 변경 및 행을 열로 변경하기
x_test = np.array(x_test).reshape([-1,1])
y_test = np.array(y_test)

In [ ]:
# 데이터 확인하기
print(x_train)
print(y_train)

#### (2) 로지스틱 회귀 학습 하기

In [ ]:
#로지스틱 회귀 라이브러리 불러오기
from sklearn.linear_model import LogisticRegression

In [ ]:
# 로지스틱 회귀 생성하기
logi_reg = LogisticRegression()

In [ ]:
# 학습하기
logi_reg.fit(x_train, y_train)

In [ ]:
# 역산을 위한 기울기와 절편이 있는지 확인하기
print('coef:', logi_reg.coef_)
print('intercept:', logi_reg.intercept_)

#### (3) 로지스틱 회귀 그래프 만들기


In [ ]:
# 기울기와 절편을 수동으로 결과 만들기
odd = [] #
for i in x_train:
    odd.append((logi_reg.coef_* i) + logi_reg.intercept_)

sigmoid_y= sigmoid(np.array(odd))
sigmoid_y = sigmoid_y.reshape(-1,1)

In [ ]:
# 역산된 그래프 표시하기
plt.scatter(x_train, y_train,color='red')
plt.plot(np.array(x_train), sigmoid_y, color='blue')
plt.rcParams['axes.grid'] = True
plt.yticks([0,0.5,1])
plt.ylim([-0.1, 1.1]) # y축의 범위: [Ymin, Ymax]
plt.show()

### 3) 로지스틱 회귀와 선형회귀 비교하기

#### (1) 선형회귀 모델링하기

In [ ]:
# 선형회귀 함수 불러오기
from sklearn.linear_model import LinearRegression
# 선형회귀 생성하기
lr = LinearRegression()
# 학습하기
lr.fit(x_train,y_train)

In [ ]:
# 수식 완성을 위한 coef 와 intercept 확인하기
print('intercept:', lr.intercept_)
print('coef:', lr.coef_)

#### (2) 선형회귀 모델과 로지스틱 회귀 모델 비교하기

In [ ]:
coef_intercept = x_train * lr.coef_ + lr.intercept_

In [ ]:
plt.scatter(x_train, y_train,color='red')
plt.plot(np.array(x_train), coef_intercept, color='green')
plt.plot(np.array(x_train), sigmoid_y, color='blue')
plt.rcParams['axes.grid'] = True
plt.show()
print('x_test:', x_test, 'y_test:', y_test)
print('로지스틱 회귀 정확도: ', logi_reg.score(x_test,y_test))
print('선형 회귀 정확도: ',lr.score(x_test,y_test))

### [확인 문제]

In [ ]:
# 문제 01






---------------------

## <a name="의사결정나무">의사결정나무(Decision tree)</a>

- sklearn.tree : https://scikit-learn.org/stable/api/sklearn.tree.html
- DecisionTreeClassifier : https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

### 2) 의사결정나무 실습하기 (DecisionTreeClassifier)

#### (1) 데이터 준비하기

In [ ]:
# 라이브러리 불러오기 (numpy, pandas, train_test_split)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# 데이터 불러오기
df = pd.read_csv("국민건강보험공단_건강검진정보_20211229.CSV", encoding = 'cp949')
# 트리 예시를 만들기 위해서 일부 특성만 추출
sample_df = df[['신장(5Cm단위)','성별코드', '체중(5Kg 단위)','음주여부']]

In [ ]:
# 샘플 데이터 중 상위 10개 추출하기
sample_df[:10]

#### (2) 데이터 전처리하기

In [ ]:
# info 정보로 결측치(Null) 확인하기
print("Info 정보 확인")
sample_df.info()

In [ ]:
# 100만개 데이터 중 결측치 196개, 결측치가 포함된 행 삭제하기
sample = sample_df.dropna()

In [ ]:
# 결측치(Null) 다시 확인하기
print("Drop 후 Info 정보 확인")
sample.info()

In [ ]:
# 원-핫 인코딩을 위해 데이터를 object 형태로 변경하기
sample = sample.astype('str')

# 원하는 타입만 변경할 경우
# sample['음주여부'] = sample['음주여부'].astype('str')
# sample = sample.astype({'음주여부':'str'})

In [ ]:
sample.info()

In [ ]:
# label(결과, Y) 생성하기
y=sample.음주여부

In [ ]:
# 음주 여부 학습 데이터 구성하기
X=sample.drop('음주여부', axis=1)

In [ ]:
# label 데이터의 편향성 확인하기
y.value_counts()

In [ ]:
# 학습/검증 데이터 분리하기
x_train, x_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    shuffle=True,
    random_state=34
)

#### (3) 의사결정나무 모델링하기

In [ ]:
# 의사결정나무 모델 불러오기
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# 의사결정나무 모델 생성하기
dt = DecisionTreeClassifier(random_state = 1001,
                            max_depth=2
                           )

In [ ]:
# 의사결정나무 학습하기
dt_model = dt.fit(x_train, y_train)

In [ ]:
# 학습 데이터 정확도 확인하기
print("학습 정확도 = ", dt_model.score(x_train, y_train))

# 검증 데이터 정확도 확인하기
print("검증 정확도 = ", dt_model.score(x_valid, y_valid))

#### (4) 의사결정나무 그래프 그리기

In [ ]:
# 그래프 라이브러리(matplotlib.pyplot) 불러오기
import matplotlib.pyplot as plt

In [ ]:
# 트리 모양 그래프 작성을 위한 라이브러리 설치하기(코랩에는 이미 설치됨)
#!pip install graphviz

In [ ]:
# 그래프 라이브러리(grphviz) 불러오기
import graphviz
# 사이킷런의 graphviz 지원 모듈 불러오기
from sklearn.tree import export_graphviz

# 그래프 생성하기
tree_graph = graphviz.Source(export_graphviz(dt_model,
                                class_names=["X","O"],
                                feature_names=[ 'height', 'sex','weight'],
                                impurity=True,
                                filled=True))

In [ ]:
# 그래프 출력하기
# (참고) 에러 발생시 사이트 참조 : https://highschoolfree.tistory.com/76
tree_graph

In [ ]:
# 그래프 파일 저장하기
tree_graph.render('tree_depth5', format="png")

### 3) 불순도 알아보기

In [ ]:
# 불순도 함수 생성하기
def gini(x):




    return gini

In [ ]:
# 데이터 준비하기(불순도 예시)
과일바구니1 = ['사과']*9
과일바구니2 = ['사과', '바나나','사과', '바나나','바나나','바나나', '복숭아','복숭아','복숭아']
과일바구니3 = ['사과', '바나나','사과', '바나나','사과','복숭아', '복숭아','사과','복숭아']

In [ ]:
print(pd.DataFrame(과일바구니1).value_counts())
print(pd.DataFrame(과일바구니2).value_counts())
print(pd.DataFrame(과일바구니3).value_counts())
print('\n지니계수 출력-----------------')
print(round(gini(pd.DataFrame(과일바구니1).value_counts()),3))
print(round(gini(pd.DataFrame(과일바구니2).value_counts()),3))
print(round(gini(pd.DataFrame(과일바구니3).value_counts()),3))

### [확인 문제]

#### 1. 앞에서 활용했던 데이터(국민건강보험공단~) 의 (sample_df : 음주여부 예측) 컬럼을 활용하여 max_depth 파라미터를 1, 15, 30으로 놓고 각각의 모델을 만들어보고 validation score가 작게 나오는  max_dept 수치를 작성하세요.(단, random_state는 1001 고정)


In [ ]:
#for 문제 직접 리스트를 넣어서 구현하기




#### 2. max_feature 파라미터 값을 1,2,3으로 하는 각각 모델을 만들고 validation Score가 가장 높은 max_feature를 작성하세요. (단, random_state는 1001로 고정하고, max_depth에 대해 앞에서 가장 작은 max_depth 고정 후 결과 산출)

In [ ]:
# 리스트를 별도로 만들어서 max_feature 변경하고 결과 산출




---------------------------

## <a name="랜덤포레스트">랜덤포레스트(Random Forest)</a>

- sklearn.ensemble : https://scikit-learn.org/stable/api/sklearn.ensemble.html  
- RandomForestClassifier : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html  

### 2) 랜덤포레스트 실습하기

#### (1) 데이터 준비하기

In [ ]:
# tensorflow 라이브러리 설치하기
#!pip install tensorflow --user

In [ ]:
# tensorflow에서 제공하는 데이터셋 mnist 불러오기
from tensorflow.keras.datasets.mnist import load_data

In [ ]:
# load_data로 데이터 할당하기
(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
# 샘플 데이터 matplotlib 그래프로 나타내보기
import matplotlib.pyplot as plt
plt.imshow(x_train[1], cmap='Greys')
plt.show()

In [ ]:
# 손 글씨 데이터는 이미지라 3차원 행렬
print("변경 전 = ",x_train.shape)

In [ ]:
# 3차원 행렬을 2차원으로 변경하기
X_train = x_train.reshape(-1, 784)
X_test = x_test.reshape(-1, 784)

# 변경 결과 확인하기
print("변경 후 = ",X_train.shape)

#### (2) 의사결정나무 모델링하기

In [ ]:
# 필요 라이브러리 불러오기(의사결정나무, 랜덤포레스트)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# 의사결정나무 학습하기
dtc = DecisionTreeClassifier(random_state=0)
dtc.fit(X_train, y_train)

In [ ]:
# 의사결정나무 결과확인하기
acc_train_dtc = dtc.score(X_train,y_train)
acc_test_dtc = dtc.score(X_test,y_test)

print(f'''학습결과 = {acc_train_dtc},검증결과 = {acc_test_dtc} ''')

#### (3) 랜덤포레스트 모델링 및 결과 비교하기

In [ ]:
# 랜덤포레스트 학습하기
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)

In [ ]:
# 랜덤포레스트 결과 보기
acc_train_rfc = rfc.score(X_train,y_train)
acc_test_rfc = rfc.score(X_test,y_test)

# 학습 결과 수치로 출력하기
print(f"""의사결정나무: train_acc = {round(acc_train_dtc,3)}, test_acc = {round(acc_test_dtc,3)}""")
print(f"""랜덤포레스트: train_acc = {round(acc_train_rfc,3)}, test_acc = {round(acc_test_rfc,3)}""")

In [ ]:
# 랜덤포레스트 결과를 토대로 비교 그래프 그리기
import matplotlib.pyplot as plt
# x 축 정의하기
acc_list_x = ['dtc_train', 'dtc_test', 'rfc_train', 'rfc_test']
# y 축 정의
acc_list_y = [acc_train_dtc, acc_test_dtc, acc_train_rfc, acc_test_rfc]
# 막대그래프 차트 색 정의하기
colors = ['orange', 'orange' , 'blue', 'blue']
# 막대그래프 설정하기
plt.bar(acc_list_x, acc_list_y, color=colors)
# 화면 출력하기
plt.grid(True)
plt.show()

### 3) 랜덤포레스트와 의사결정나무 비교하기

In [ ]:
# 의사결정나무 결과와 동일하게 랜덤포레스트 구성하기
rft = RandomForestClassifier(
    random_state=0,
    # 나무 개수를 1개로 설정하기
    n_estimators=1,
    # max_feature는 의사결정나무 기준으로 변경하기
    max_features=None,
    # 부트스트랩 샘플링은 OFF하기
    bootstrap=False,
)
rft.fit(X_train, y_train)

In [ ]:
# 학습 결과 저장하기
acc_train_rfc = rft.score(X_train,y_train)
acc_test_rfc = rft.score(X_test,y_test)
acc_train_dtc = dtc.score(X_train,y_train)
acc_test_dtc = dtc.score(X_test,y_test)

#의사결정나무 & 랜덤포레스트가 유사한 결과를 출력하는지 비교하기
print(f"""의사결정나무: train_acc = {round(acc_train_dtc,3)}, test_acc = {round(acc_test_dtc,3)}""")
print(f"""랜덤포레스트: train_acc = {round(acc_train_rfc,3)}, test_acc = {round(acc_test_rfc,3)}""")

### [확인 문제]

#### 1. Max_features 의 값을 40, 50, 60 으로 변경해보고 가장 좋은 결과값을 작성 하세요. (단, random_state는 0으로 고정)

----------------------------------------------------

## <a name="그라디언트부스팅">그라디언트 부스팅(gradient boosting)</a>

- GradientBoostingClassifier : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html   

### 2) 그라디언트 부스팅 실습하기

#### (1) 데이터 준비하기

In [ ]:
# tensorflow에서 데이터 불러오기
from tensorflow.keras.datasets.mnist import load_data
# 그래프 라이브러리 불러오기
import matplotlib.pyplot as plt

In [ ]:
# 데이터 불러오기
(x_train, y_train), (x_test, y_test) = load_data()

# 학습시간 고려해서 2000건 데이터만 사용하기
x_train = x_train[:2000]
y_train = y_train[:2000]
x_test = x_test[:2000]
y_test = y_test[:2000]

In [ ]:
# 샘플 데이터 확인하기
plt.imshow(x_train[7], cmap='Greys')
plt.show()

In [ ]:
# 샘플 데이터 확인하기
plt.imshow(x_train[1], cmap='Greys')
plt.show()

In [ ]:
# 학습을 위한 2차원 행렬로 변경하기
X_train = x_train.reshape(-1, 784)
X_test = x_test.reshape(-1, 784)

#### (2) 알고리즘 별 학습 및 결과 비교하기

In [ ]:
# 의사결정나무, 랜덤포레스트, 그라디언트부스팅 라이브러리 불러오기
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# 의사결정나무 불러오기 및 학습하기
dtc = DecisionTreeClassifier(random_state=0)
dtc.fit(X_train, y_train)

In [ ]:
# 의사결정나무 학습 결과 저장하기
acc_train_dtc = dtc.score(X_train,y_train)
acc_test_dtc = dtc.score(X_test,y_test)

In [ ]:
# 랜덤포레스트 불러오기 및 학습하기
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)

In [ ]:
# 랜덤포레스트 학습 결과 저장하기
acc_train_rfc = rfc.score(X_train,y_train)
acc_test_rfc = rfc.score(X_test,y_test)

In [ ]:
# 그라디언트부스팅 불러오기 및 학습하기
gbc = GradientBoostingClassifier(random_state=0, verbose=1)
gbc.fit(X_train, y_train)

In [ ]:
# 그라디언트부스팅 학습 결과 저장하기
acc_train_gbc = gbc.score(X_train,y_train)
acc_test_gbc = gbc.score(X_test,y_test)

# 각 알고리즘별 성능 비교하기
print(f"""의사결정나무: train_acc = {round(acc_train_dtc,3)}, test_acc = {round(acc_test_dtc,3)}""")
print(f"""랜덤포레스트: train_acc = {round(acc_train_rfc,3)}, test_acc = {round(acc_test_rfc,3)}""")
print(f"""그라디언트부스팅: train_acc = {round(acc_train_gbc,3)}, test_acc = {round(acc_test_gbc,3)}""")

In [ ]:
# 비교 그래프 그리기
import matplotlib.pyplot as plt
acc_list_x = ['dtc_train', 'dtc_test', 'rfc_train', 'rfc_test', 'gbc_train', 'gbc_test', ]
acc_list_y = [acc_train_dtc, acc_test_dtc, acc_train_rfc, acc_test_rfc, acc_train_gbc, acc_test_gbc]
colors = ['orange', 'orange' , 'blue', 'blue', 'red', 'red']
plt.bar(acc_list_x,acc_list_y, color=colors)
plt.ylim([0.8,1.0])
plt.grid(True)
plt.show()

### [확인 문제]

#### 1. 그래디언트 부스팅 알고리즘으로 n_estimator를 200으로 설정(기본값은 100)하여 학습시켜 보고 기존 그래디언트 부스팅 결과와 비교하는 그래프 코드를 작성하세요.

In [ ]:
# 그라디언트부스팅 불러오기 및 학습하기





In [ ]:
# 그라디언트부스팅 학습 결과 다시 저장하기
acc_train_gbc_200 = gbc_200.score(X_train,y_train)
acc_test_gbc_200 = gbc_200.score(X_test,y_test)

print(f"""그라디언트부스팅: train_acc = {round(acc_train_gbc,3)}, test_acc = {round(acc_test_gbc,3)}""")
print(f"""튜닝 그라디언트부스팅: train_acc = {round(acc_train_gbc_200,3)}, test_acc = {round(acc_test_gbc_200,3)}""")

In [ ]:
import matplotlib.pyplot as plt
acc_list_x = ['gbc_train', 'gbc_test', 'tune_gbc_train', 'tune_gbc_test', ]
acc_list_y = [acc_train_gbc, acc_test_gbc, acc_train_gbc_200, acc_test_gbc_200]
colors = ['orange', 'orange' , 'blue', 'blue']
plt.bar(acc_list_x, acc_list_y, color=colors)
plt.ylim([0.8,1.0])
plt.show()